<a href="https://colab.research.google.com/github/G3orgeBS/imersao_alura/blob/main/ChatBot_de_Primeiros_Socorros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Instalar Bibliotecas

!pip install -q google-generativeai langchain langchain-google-genai faiss-cpu langchain-community
print("Bibliotecas instaladas!")


import os
import google.generativeai as genai

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

GOOGLE_API_KEY = "GOOGLE_API_KEY"

if not GOOGLE_API_KEY or GOOGLE_API_KEY == "GOOGLE_API_KEY":
    print("ERRO: Por favor, configure sua GOOGLE_API_KEY do Gemini.")

else:
    genai.configure(api_key=GOOGLE_API_KEY)
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print("Bibliotecas importadas e API Key configurada (verifique se a chave foi inserida).")

Bibliotecas instaladas!
ERRO: Por favor, configure sua GOOGLE_API_KEY do Gemini.
Bibliotecas importadas e API Key configurada (verifique se a chave foi inserida).


In [6]:
# Base de Conhecimento do Chatbot de Primeiros Socorros

base_conhecimento = {
    'corte': {
        'descricao_inicial': 'Ok, vamos lidar com um corte. É importante avaliar a gravidade.',
        'perguntas': [
            {'pergunta': 'O sangramento é muito intenso, em jato ou não para após pressionar firmemente por 5-10 minutos?',
             'respostas': {
                 'sim': 'Sangramento intenso e incontrolável é uma emergência! Aplique pressão direta firme com um pano limpo ou gaze e PROCURE ATENÇÃO MÉDICA IMEDIATA ou ligue para o serviço de emergência.',
                 'nao': 'Bom. Continue pressionando firmemente. Se diminuir, o próximo passo é limpar.'
             }
            },
            {'pergunta': 'O corte é profundo (você vê músculos ou ossos), muito grande, ou tem bordas irregulares que não se juntam facilmente?',
             'respostas': {
                 'sim': 'Cortes profundos, grandes ou com bordas irregulares podem precisar de sutura (pontos). Após controlar o sangramento, PROCURE UM MÉDICO ou vá a uma unidade de saúde.',
                 'nao': 'Ok. Parece ser um corte mais superficial. Lave o corte gentilmente com água corrente limpa e sabão neutro.'
             }
            },
             {'pergunta': 'Após lavar, você removeu sujeiras visíveis (terra, vidro pequeno, etc.)?',
             'respostas': {
                 'sim': 'Ótimo. Se necessário, lave novamente. Mantenha a área limpa.',
                 'nao': 'Se houver sujeira visível, tente removê-la cuidadosamente com água corrente. Se não sair, não esfregue e PROCURE UM MÉDICO.'
             }
            }
        ],
        'orientacao_final': 'Após limpar e estancar o sangramento, se aplicável, seque a área ao redor e cubra o corte com um curativo limpo e seco (band-aid, gaze e fita). Troque o curativo diariamente e sempre que ficar molhado ou sujo. Observe por sinais de infecção (vermelhidão aumentada, inchaço, dor latejante, pus, febre). Se aparecerem, procure um médico.'
    },
    'queimadura': {
         'descricao_inicial': 'Entendido, uma queimadura. É crucial determinar o grau e a extensão da queimadura.',
         'perguntas': [
             {'pergunta': 'É uma queimadura grande (maior que a palma da mão da vítima), atinge rosto/mãos/pés/genitais, ou a pele está esbranquiçada, marrom escura ou carbonizada (sem dor ou com pouca dor)?',
              'respostas': {
                  'sim': 'Estas são queimaduras de 2º grau profundo ou 3º grau. São emergências! Chame o serviço de emergência ou PROCURE ATENÇÃO MÉDICA URGENTE IMEDIATAMENTE. NÃO remova roupas grudadas e NÃO aplique nada na queimadura.',
                  'nao': 'Ok, parece ser uma queimadura de 1º grau (apenas vermelhidão e dor) ou 2º grau superficial (vermelhidão, dor e bolhas pequenas). Continue...'
              }
             },
              {'pergunta': 'A queimadura tem bolhas que cobrem uma área maior que uma moeda de 1 real?',
              'respostas': {
                  'sim': 'Queimaduras com bolhas significativas (2º grau superficial) precisam de cuidado. NÃO fure as bolhas. Resfrie a área com água corrente fria (NÃO gelada) por 10-20 minutos.',
                  'nao': 'Ok, parece uma queimadura de 1º grau (apenas vermelhidão). Resfrie a área afetada com água corrente fria (NÃO gelada) por 10-20 minutos.'
              }
             },
              {'pergunta': 'Você já resfriou a área com água fria por 10-20 minutos?',
              'respostas': {
                  'sim': 'Excelente. Isso ajuda a aliviar a dor e limitar o dano. NÃO aplique gelo diretamente, manteiga, pasta de dente, café, etc.',
                  'nao': 'Por favor, resfrie a área queimada com água corrente fria (NÃO gelada) por 10-20 minutos AGORA. Isso é o passo mais importante para queimaduras leves.'
              }
             }
         ],
         'orientacao_final': 'Após resfriar, se a pele não estiver com bolhas grandes ou abertas, você pode cobrir a área com um pano limpo e seco ou um curativo próprio para queimaduras, sem apertar. Use analgésicos comuns para a dor, se necessário. NUNCA aplique cremes, pomadas caseiras ou produtos sem orientação médica em queimaduras. Se a dor piorar, surgirem sinais de infecção (vermelhidão, inchaço, pus) ou se a queimadura não melhorar em poucos dias, procure um médico.'
    },
     'intoxicacao': {
        'descricao_inicial': 'Intoxicação pode ser grave. Mantenha a calma e identifique a substância, se possível.',
        'perguntas': [
            {'pergunta': 'A pessoa está inconsciente, com dificuldade grave para respirar, tendo convulsões ou não reage?',
             'respostas': {
                 'sim': 'Esta é uma emergência médica gravíssima! Chame o serviço de emergência IMEDIATAMENTE. Se a pessoa não estiver respirando ou sem pulso, inicie RCP se souber.',
                 'nao': 'Ok. A pessoa está consciente e respirando. Tente descobrir QUAL substância foi ingerida, inalada ou entrou em contato com a pele/olhos e QUANDO isso aconteceu.'
             }
            },
             {'pergunta': 'Você já entrou em contato com um Centro de Intoxicações ou serviço de emergência para obter orientação específica para esta substância?',
             'respostas': {
                 'sim': 'Siga rigorosamente as instruções dadas pelo Centro de Intoxicações ou serviço de emergência. Eles têm o conhecimento específico sobre a substância.',
                 'nao': 'Ligue para um Centro de Intoxicações (ex: CIT/PR 0800 410 148 - número de referência no Brasil, procure o da sua região) ou serviço de emergência AGORA. Tenha a embalagem da substância (ou o máximo de informação possível) em mãos. NÃO induza o vômito a menos que seja EXPLICITAMENTE instruído por um profissional de saúde.'
             }
            }
        ],
        'orientacao_final': 'NUNCA induza o vômito sem orientação médica ou de um Centro de Intoxicações. Mantenha a vítima sob observação constante. Se a substância entrou em contato com a pele ou olhos, lave abundantemente com água por pelo menos 15-20 minutos. Leve a embalagem da substância para o serviço médico. Mesmo que a pessoa pareça bem, a avaliação médica pode ser necessária.'
    },
     'picada_serpente': {
        'descricao_inicial': 'Picada de serpente peçonhenta é uma emergência médica! Mantenha a calma da vítima.',
        'perguntas': [
            {'pergunta': 'Você já chamou o serviço de emergência (ex: SAMU 192, Bombeiros 193) ou está levando a vítima URGENTEMENTE para o hospital mais próximo com soro antiveneno disponível?',
             'respostas': {
                 'sim': 'Excelente. O tratamento com soro antiveneno é o mais importante e deve ser feito o mais rápido possível em ambiente hospitalar.',
                 'nao': 'CHAME o serviço de emergência IMEDIATAMENTE ou transporte a vítima com urgência para um hospital preparado para atender acidentes com serpentes. O TEMPO é crucial!'
             }
            },
             {'pergunta': 'Enquanto aguarda socorro, a vítima está em repouso e a área picada está sendo mantida em posição mais baixa que o coração (se possível)?',
             'respostas': {
                 'sim': 'Correto. Manter a calma, evitar movimentos e manter a área picada baixa ajuda a retardar a circulação do veneno.',
                 'nao': 'Peça para a vítima ficar o mais calma possível e em repouso. Se a picada foi em um membro, mantenha-o em posição mais baixa que o coração, se isso for confortável.'
             }
            },
            {'pergunta': 'Você limpou o local da picada suavemente com água e sabão?',
             'respostas': {
                 'sim': 'Bom. Mantenha o local limpo e seco. Não aplique nada sobre a picada (pó de café, álcool, querosene, etc.).',
                 'nao': 'Lave o local da picada suavemente com água e sabão, se possível. Isso ajuda a prevenir infecções secundárias.'
             }
            }
        ],
        'orientacao_final': 'NUNCA FAÇA: torniquete/garrote (pode piorar a circulação e causar necrose), cortes no local da picada, sugar o veneno, aplicar gelo ou calor, ou dar bebidas alcoólicas. Tente identificar a serpente (tire uma foto à distância com segurança, se possível) para ajudar no diagnóstico, mas NÃO arrisque outra picada. O mais importante é o transporte rápido para o hospital para aplicação do soro.'
    },
     'engasgo': {
        'descricao_inicial': 'Engasgo acontece quando algo bloqueia as vias aéreas. Avalie a situação rapidamente.',
        'perguntas': [
            {'pergunta': 'A pessoa consegue tossir FORTE, falar ou respirar?',
             'respostas': {
                 'sim': 'Engasgo parcial. NÃO INTERFIRA. Incentive a pessoa a CONTINUAR TOSSINDO FORTE. A tosse é a maneira mais eficaz de expelir o objeto.',
                 'nao': 'Engasgo grave, a pessoa não consegue tossir, falar ou respirar. É uma emergência!'
             }
            },
             {'pergunta': 'A pessoa engasgada está CONSCIENTE e de pé/sentada?',
             'respostas': {
                 'sim': 'Ótimo. Se a tosse não for eficaz ou não houver tosse, inicie as manobras de desengasgo (Manobra de Heimlich).',
                 'nao': 'Se a pessoa engasgada estiver inconsciente, CHAME o serviço de emergência IMEDIATAMENTE e inicie compressões torácicas (RCP) se souber, verificando a boca por objetos antes das ventilações.'
             }
            }
        ],
        'orientacao_final': 'Para engasgo grave em adulto CONSCIENTE: Fique atrás da vítima, abrace-a na altura do abdômen (entre o umbigo e o final do esterno), feche uma mão em punho e coloque sobre o abdômen, cubra com a outra mão. Faça compressões rápidas para DENTRO e para CIMA até que o objeto seja expelido, a pessoa comece a tossir/respirar ou perca a consciência. Se perder a consciência, deite-a no chão e CHAME ajuda, iniciando RCP se souber.'
    },
     'fratura': {
        'descricao_inicial': 'Suspeita de fratura (osso quebrado). NÃO mova a área afetada desnecessariamente.',
        'perguntas': [
            {'pergunta': 'Há sangramento intenso, o osso está visível, ou a deformidade no membro é óbvia e grave?',
             'respostas': {
                 'sim': 'Isto pode ser uma fratura exposta ou grave. CONTROLE o sangramento com pressão direta (cuidado para não mover o osso) e CHAME o serviço de emergência IMEDIATAMENTE ou transporte para o hospital.',
                 'nao': 'Ok. Parece ser uma fratura fechada ou menos grave, mas ainda assim requer atenção médica.'
             }
            },
             {'pergunta': 'Você imobilizou o membro afetado para que ele não se mova (usando talas improvisadas com panos, jornais enrolados, etc. e amarrando acima e abaixo da lesão)?',
             'respostas': {
                 'sim': 'Correto. Manter o membro imóvel ajuda a prevenir mais lesões e reduzir a dor.',
                 'nao': 'NÃO MOVA o membro afetado. Imobilize a área lesionada na posição em que a encontrou, usando materiais rígidos (tábuas, revistas grossas) e amarrando com faixas ou tiras de pano sem apertar demais. Imobilize a articulação acima e abaixo da suspeita de fratura.'
             }
            }
        ],
        'orientacao_final': 'Após imobilizar, aplique compressas frias (gelo envolto em pano) na área para reduzir inchaço e dor. Mantenha o membro elevado, se possível e confortável. NUNCA tente colocar o osso no lugar. PROCURE UM MÉDICO ou vá a um pronto-socorro o mais rápido possível. A avaliação médica com raio-x é essencial para confirmar a fratura e o tratamento correto.'
    },
    'desmaio': {
        'descricao_inicial': 'Desmaio (perda temporária da consciência) geralmente ocorre por falta temporária de fluxo sanguíneo para o cérebro.',
        'perguntas': [
            {'pergunta': 'A pessoa recuperou a consciência rapidamente (em 1-2 minutos) após ser deitada e ter as pernas elevadas?',
             'respostas': {
                 'sim': 'Ótimo. O desmaio provavelmente foi simples. Mantenha a pessoa deitada por mais alguns minutos. Levante-a devagar. Ofereça água.',
                 'nao': 'Se a pessoa não recuperar a consciência rapidamente (em 1-2 minutos) ou se demorar, isso pode ser mais sério. CHAME o serviço de emergência IMEDIATAMENTE.'
             }
            },
             {'pergunta': 'Durante o desmaio ou após recuperar a consciência, a pessoa apresentou convulsões, dificuldade para respirar, dor no peito, dor de cabeça forte, perda de controle da bexiga/intestino, ou não consegue mover um lado do corpo?',
             'respostas': {
                 'sim': 'Estes são sinais de ALERTA de um problema mais sério. CHAME o serviço de emergência IMEDIATAMENTE.',
                 'nao': 'Bom que estes sinais não estão presentes. Provavelmente foi um desmaio simples. Continue as orientações.'
             }
            }
        ],
        'orientacao_final': 'Se a pessoa desmaiar, deite-a de costas (se não houver suspeita de lesão na coluna) e eleve as pernas cerca de 30 cm acima do nível do coração. Afrouxe roupas apertadas (colarinho, cinto). Mantenha as vias aéreas livres. Se ela acordar, mantenha-a deitada por um tempo. Se não acordar rapidamente, se machucou na queda, estiver grávida, diabética, ou apresentar qualquer um dos sinais de alerta mencionados, PROCURE AJUDA MÉDICA URGENTE.'
    },
     'convulsao': {
        'descricao_inicial': 'Uma convulsão é uma atividade elétrica anormal no cérebro. O mais importante é proteger a pessoa de se machucar durante a crise.',
        'perguntas': [
            {'pergunta': 'A pessoa está tendo movimentos musculares involuntários, caiu no chão e não responde?',
             'respostas': {
                 'sim': 'É uma convulsão. Mantenha a calma. O foco é proteger a pessoa DURANTE a crise.',
                 'nao': 'Se a pessoa não tem movimentos involuntários generalizados e está responsiva, não é uma convulsão típica. Avalie outros sintomas ou procure ajuda médica.'
             }
            },
             {'pergunta': 'Você removeu objetos perigosos próximos à pessoa (móveis, objetos duros/pontiagudos) e colocou algo macio sob a cabeça dela (travesseiro, casaco dobrado)?',
             'respostas': {
                 'sim': 'Ótimo. Proteger a cabeça e evitar lesões é fundamental durante a convulsão.',
                 'nao': 'Afaste rapidamente objetos que possam machucar a pessoa durante a convulsão. Coloque algo macio e plano sob a cabeça para protegê-la.'
             }
            },
             {'pergunta': 'Você tentou segurar ou conter os movimentos da pessoa?',
             'respostas': {
                 'sim': 'NÃO TENTE SEGURAR ou conter os movimentos da pessoa durante a convulsão. Isso não ajuda e pode causar lesões nela ou em você.',
                 'nao': 'Correto. NUNCA segure ou tente conter os movimentos convulsivos. Deixe a pessoa se mover.'
             }
            },
             {'pergunta': 'Você colocou ou tentou colocar algo na boca da pessoa para "não engolir a língua"?',
             'respostas': {
                 'sim': 'NUNCA COLOQUE NADA NA BOCA de alguém tendo uma convulsão! Isso pode causar ferimentos graves na boca, dentes quebrados ou bloquear as vias aéreas.',
                 'nao': 'Correto. NUNCA coloque nada na boca da pessoa durante uma convulsão.'
             }
            }
        ],
        'orientacao_final': 'A maioria das convulsões para sozinha em poucos minutos. Cronometre a duração. Após a convulsão parar, vire a pessoa GENTILMENTE de lado (posição de recuperação) para ajudar a manter as vias aéreas abertas, pois ela pode estar inconsciente ou sonolenta. Fique com ela até que se recupere completamente. CHAME o serviço de emergência se a convulsão durar mais de 5 minutos, se a pessoa tiver dificuldade para respirar após a crise, se for a primeira convulsão dela, se ela se machucou durante a crise, se for diabética, grávida, ou se não recuperar a consciência após alguns minutos.'
    },
    'ajuda': {
        'descricao_inicial': 'Posso ajudar com os seguintes tópicos de primeiros socorros básicos. Por favor, digite o nome do tópico:\ncorte\nqueimadura\nintoxicacao\npicada_serpente\nengasgo\nfratura\ndesmaio\nconvulsao\nDigite "sair" para encerrar.',
        'perguntas': [],
        'orientacao_final': ''
    },
    'sair': {
        'descricao_inicial': 'Saindo do chatbot. Lembre-se: este bot oferece apenas orientações básicas e NÃO substitui atendimento médico profissional. Em caso de emergência, procure ajuda IMEDIATAMENTE.',
        'perguntas': [],
        'orientacao_final': ''
    }
}

print("Base de conhecimento de primeiros socorros carregada com detalhes.")

Base de conhecimento de primeiros socorros carregada com detalhes.


In [7]:
# Lógica de Interação do Chatbot

print("Olá! Sou um chatbot de primeiros socorros básicos.")
print("Posso oferecer orientações para situações comuns baseadas em protocolos simples.")
print("Digite 'ajuda' para ver os tópicos que conheço ou 'sair' para encerrar.")
print("-" * 60) # Separador

while True:
    # Pede a entrada do usuário para escolher um tópico
    entrada_usuario = input("Em que posso ajudar? (digite o tópico, 'ajuda' ou 'sair'): ").lower().strip()

    # Processa a entrada do usuário
    if entrada_usuario == 'sair':
        print(base_conhecimento['sair']['descricao_inicial'])
        break # Sai do loop while

    elif entrada_usuario == 'ajuda':
         print(base_conhecimento['ajuda']['descricao_inicial'])
         print("-" * 60) # Separador
         continue # Volta para o início do loop para pedir nova entrada

    # Verifica se o tópico digitado existe na base de conhecimento
    elif entrada_usuario in base_conhecimento:
        situacao_atual = base_conhecimento[entrada_usuario]
        # Exibe o título e a descrição inicial do tópico escolhido
        print(f"\n--- {entrada_usuario.replace('_', ' ').title()} ---") # Título formatado
        print(situacao_atual['descricao_inicial'])

        # Itera sobre as perguntas definidas para a situação
        for i, passo in enumerate(situacao_atual.get('perguntas', [])):
            # Loop interno para garantir que a resposta seja 'sim' ou 'nao' válida
            while True:
                # Exibe a pergunta ao usuário
                resposta_usuario = input(f"Pergunta {i+1}: {passo['pergunta']} (sim/nao): ").lower().strip()

                # Verifica se a resposta do usuário é uma das opções esperadas ('sim' ou 'nao')
                if resposta_usuario in passo['respostas']:
                    # Imprime a resposta correspondente da base de conhecimento
                    print(passo['respostas'][resposta_usuario])
                    # Sai do loop interno de validação de resposta para ir para a próxima pergunta ou finalização
                    break
                else:
                    # Mensagem de erro para resposta inválida
                    print("Desculpe, não entendi sua resposta. Por favor, responda 'sim' ou 'nao'.")

        # Após passar por todas as perguntas, imprime a orientação final, se existir
        if situacao_atual.get('orientacao_final'):
             print(situacao_atual['orientacao_final'])

        print("-" * 60) # Separador para a próxima interação

    # Se o tópico digitado não for reconhecido
    else:
        print("Desculpe, não consigo ajudar com este tópico específico.")
        print("Digite 'ajuda' para ver a lista de tópicos que conheço.")
        print("-" * 60)

print("Chatbot encerrado.")

Olá! Sou um chatbot de primeiros socorros básicos.
Posso oferecer orientações para situações comuns baseadas em protocolos simples.
Digite 'ajuda' para ver os tópicos que conheço ou 'sair' para encerrar.
------------------------------------------------------------
Em que posso ajudar? (digite o tópico, 'ajuda' ou 'sair'): ajuda
Posso ajudar com os seguintes tópicos de primeiros socorros básicos. Por favor, digite o nome do tópico:
corte
queimadura
intoxicacao
picada_serpente
engasgo
fratura
desmaio
convulsao
Digite "sair" para encerrar.
------------------------------------------------------------
Em que posso ajudar? (digite o tópico, 'ajuda' ou 'sair'): engasgo

--- Engasgo ---
Engasgo acontece quando algo bloqueia as vias aéreas. Avalie a situação rapidamente.
Pergunta 1: A pessoa consegue tossir FORTE, falar ou respirar? (sim/nao): nao
Engasgo grave, a pessoa não consegue tossir, falar ou respirar. É uma emergência!
Pergunta 2: A pessoa engasgada está CONSCIENTE e de pé/sentada? (si